# Module 4: Hyperparameters

In [1]:
# Setup the matplotlib styling
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import pandas as pd
import numpy as np

try:
    # Try to use the BI style sheet for plots
    plt.style.use('matplotlibrc')
    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=[(136/256, 76/256, 255/256), (60/256, 170/256, 207/256), (12/256, 229/256, 177/256)]) 
    
    colors = [(0.53125, 0.296875, 0.99609375), (0.453125, 0.3984375, 0.9453125), (0.375, 0.4921875, 0.89453125), (0.3046875, 0.578125, 0.8515625), (0.234375, 0.6640625, 0.80859375), (0.16015625, 0.75390625, 0.76171875), (0.09375, 0.8359375, 0.72265625), (0.046875, 0.89453125, 0.69140625), (0.0, 0.875, 0.6640625)]
    bicmap = LinearSegmentedColormap.from_list(name='BIcmp', 
                                                colors=colors,
                                                N=len(colors))
    cm_bright = ListedColormap([(0.53125, 0.296875, 0.99609375), (12/256, 229/256, 177/256)])
except:
    bicmap = plt.cm.BuGn 
    colors = ['r', 'g', 'b']

## **Exercise 4.1:**
Fill in the blanks in the following sentences:

In the case of underfitting, the ``Training`` error is high.  
In the case of overfitting, the ``Training`` error is low but the ``Test`` error is high.  

## **Exercise 4.2:**

You are a Data Scientist in an advertising company. Given a single continuous parameter x, you have to predict a continuous target variable y. You decided to use a polynomial regression model (remember that you can use make_pipeline from sklearn to do that if you want). 

### **Exercise 4.2.1:** 
Write two functions in python:

fit_model(x, y, degree)
Given a a single feature ‘x’ and a target variable ‘y’ and the degree for the polynomial regression, this function should compute the actual model and return it.

evaluate_model(model, x, y)
Given a model and a dataset with a single feature ‘x’ and a target variable ‘y’, this functions should compute the MAE (Mean Absolute Error) of the model and return it.

In [2]:
# Import necessary modules
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

def fit_model(x, y, degree):
    poly_reg = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_reg.fit(x, y)
    return poly_reg

def evaluate_model(model, x, y):
    return mean_absolute_error(y, model.predict(x))

### **Exercise 4.2.2:** 

For each value of the hyperparameter ‘d’ in 1,2,3,…,20,
1. Train a model on the given data in train.csv
2. Evaluate the model on test.csv

Which value for the hyperparameter ‘d’ is best suited for putting the model into production?


In [3]:
train = pd.read_csv('data/train.csv')
train_x = train[['x']].values
train_y = train['y'].values
train.head()

,x,y
0,0.000000,0.000000
1,0.392699,0.882683
2,0.785398,0.707107
3,1.570796,1.000000
4,1.963495,1.423880


In [4]:
test = pd.read_csv('data/test.csv')
test_x = test[['x']].values
test_y = test['y'].values
test.head()

,x,y
0,1.178097,0.423880
1,2.748894,-0.117317
2,4.319690,-1.423880
3,5.890486,-0.882683


In [5]:
maes = []
for d in range(1, 20+1):
    model = fit_model(train_x, train_y, d)
    mae = evaluate_model(model, test_x, test_y)
    print(f'Degree {d} with error: {mae:0.3f}')
    maes.append(mae)

# Outputs the rank of each degree
dict(zip(list(range(1, 21)), np.argsort(np.argsort(np.array(maes)))))

Degree 1 with error: 0.528
Degree 2 with error: 0.537
Degree 3 with error: 0.641
Degree 4 with error: 0.650
Degree 5 with error: 0.658
Degree 6 with error: 0.673
Degree 7 with error: 0.561
Degree 8 with error: 0.608
Degree 9 with error: 0.683
Degree 10 with error: 1.168
Degree 11 with error: 0.922
Degree 12 with error: 2.334
Degree 13 with error: 2.386
Degree 14 with error: 0.518
Degree 15 with error: 6.280
Degree 16 with error: 21.358
Degree 17 with error: 27.736
Degree 18 with error: 158.650
Degree 19 with error: 1580.745
Degree 20 with error: 799.845


{1: 1,
 2: 2,
 3: 5,
 4: 6,
 5: 7,
 6: 8,
 7: 3,
 8: 4,
 9: 9,
 10: 11,
 11: 10,
 12: 12,
 13: 13,
 14: 0,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 19,
 20: 18}

The best degree is 14!

### **Exercise 4.2.3:**

In [6]:
val = pd.read_csv('data/val.csv')
val_x = val[['x']].values
val_y = val['y'].values
val.head()

,x,y
0,0.000000,-0.011248
1,0.006289,0.101211
2,0.012579,0.106121
3,0.018868,-0.103820
4,0.025158,0.070877



Since there are 1000 ad requests coming in each hour, the hourly company profit for a real-world application is computed as 1000*(1-MAE) where MAE is the mean average error.

1.	Retrain the model on the whole dataset using this best suitable degree d. We simulate that you have put the model into production by providing a lot of real-world data (val.csv) that comes in after putting the model in production.
What is the company profit (hourly and yearly) of that model on the real-world data?

In [7]:
# Create the whole dataset
x = np.append(train_x, test_x, axis=0)
y = np.append(train_y, test_y)

In [8]:
# Retrain the model on the chosen degree
model_14 = fit_model(x, y, 14)

In [9]:
# Evaluate the model on the validation dataset
mae_14 = evaluate_model(model_14, val_x, val_y)
mae_14

0.3894963698983938

In [10]:
# Calculate the profit of the model
profit_14 = 1000 * (1-mae_14)
profit_14

610.5036301016063

2.	Retrain the model with degree d=3 on the whole dataset and evaluate it on val.csv. What is the company profit (hourly and yearly) of that model on the real-world data?

In [11]:
# Retrain the model with degree 3
model_3 = fit_model(x, y, 3)

In [12]:
# Evaluate the model
mae_3 = evaluate_model(model_3, val_x, val_y)
mae_3

0.13178017956437377

In [13]:
# Calculate the profits of the model
profit_3 = 1000 * (1-mae_3)
profit_3

868.2198204356262

3.	Explain your finding and decide which model should be put into production.

The model with 14 degrees is underperforming in production because with our hyperparameter optimization we overfitted the test set.
If we would use the degree 3 model we would earn 250$ more per hour more.

In [14]:
250*24*365

2190000

We earn 2.190.000$ more in year with model 3.

### **Exercise 4.2.4**

Provide a method how to evaluate the performance of a hyperparameter more reliably.

The correct method would be to use leave-one-out cross validation for each degree on the combination of train and test set together.

In [15]:
# Example of correct Hyperparameter evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

maes = []
for d in range(1, 20+1):
    model = fit_model(train_x, train_y, d)
    mae = np.mean(cross_val_score(model, x, y, scoring=make_scorer(mean_absolute_error)))
    print(f'Degree {d} with error: {mae:0.3f}')
    maes.append(mae)

# Outputs the rank of each degree
dict(zip(list(range(1, 21)), np.argsort(np.argsort(np.array(maes)))))

Degree 1 with error: 0.518
Degree 2 with error: 0.775
Degree 3 with error: 0.510
Degree 4 with error: 0.911
Degree 5 with error: 1.581
Degree 6 with error: 2.143
Degree 7 with error: 17.570
Degree 8 with error: 8.460
Degree 9 with error: 149.075
Degree 10 with error: 120.874
Degree 11 with error: 495.826
Degree 12 with error: 92.322
Degree 13 with error: 5.291
Degree 14 with error: 26.400
Degree 15 with error: 69.051
Degree 16 with error: 96.792
Degree 17 with error: 48.804
Degree 18 with error: 622.664
Degree 19 with error: 17567.622
Degree 20 with error: 35045.137


{1: 1,
 2: 2,
 3: 0,
 4: 3,
 5: 4,
 6: 5,
 7: 8,
 8: 7,
 9: 15,
 10: 14,
 11: 16,
 12: 12,
 13: 6,
 14: 9,
 15: 11,
 16: 13,
 17: 10,
 18: 17,
 19: 18,
 20: 19}

Now the correct hyperparameter gets selected.

## **Bonus 1**

Let's try some automated Hyperparameter Optimization.

- Run the different HPO methods
- What is the best value you can achieve with a SVM model?
- Make some modifications to the methods to speed up the search
- What method is the most efficient for this problem?

### The problem dataset

In [16]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

x, y = make_classification(
    n_samples=200,
    n_features=20,
    n_informative=10,
    n_redundant=2,
    n_repeated=2,
    n_classes=2,
    n_clusters_per_class=4,
    flip_y=0.01,
    class_sep=0.5)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

We will try to optimize a support vector machine which has two important parameters for the parameters "C" and "gamma".

In [17]:
from sklearn.svm import SVC

svc = SVC()

### Grid Search

In [18]:
# Import necessary requirements
from sklearn.model_selection import GridSearchCV
import scipy
from sklearn.utils.fixes import loguniform

In [19]:
# Define the search space for the Grid Search
params = [
  {'C': [0.1, 1, 10, 100, 1000, 2000], 'gamma': [0.005, 0.001 , 0.0001], 'kernel': ['rbf']},
 ]

In [20]:
# Initialize GridSearch
gcv = GridSearchCV(
    estimator=svc,
    param_grid=params,
    n_jobs=-1,
)

In [21]:
# fit 
gcv.fit(x_train, y_train)

GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.1, 1, 10, 100, 1000, 2000],
                          'gamma': [0.005, 0.001, 0.0001], 'kernel': ['rbf']}])

In [22]:
# Get the parameters that achieved the best performance
gcv.best_params_

{'C': 1, 'gamma': 0.005, 'kernel': 'rbf'}

In [23]:
# Evaluate the model on unseen data
g_model = SVC(**gcv.best_params_)
g_model.fit(x_train, y_train)
g_model.score(x_test, y_test)

0.625

### Random Search

Our random search will be allowed to have as many runs as the GridSearch had.

In [24]:
num_runs = len(gcv.cv_results_['params'])

In [25]:
# Import the necessary requirements
from sklearn.model_selection import RandomizedSearchCV

In [26]:
# Define the search space
params = {
    'C': loguniform(1e-1, 2e3),
    'gamma': loguniform(1e-4, 1e-3),
    'kernel': ['rbf']
}

In [27]:
# Initialize the Random Search
rcv = RandomizedSearchCV(
    estimator=svc,
    param_distributions=params,
    n_iter=20
)

In [28]:
# Perform the fit
rcv.fit(x_train, y_train)

RandomizedSearchCV(estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb827f9a910>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb827f847c0>,
                                        'kernel': ['rbf']})

In [29]:
# Get the hyperparameters that created the best performance
rcv.best_params_

{'C': 17.370675563958468, 'gamma': 0.0007399354764248056, 'kernel': 'rbf'}

In [30]:
# Evaluate model
r_model = SVC(**rcv.best_params_)
r_model.fit(x_train, y_train)
r_model.score(x_test, y_test)

0.6

### Bayesian Optimization

Let's use a more intelligent method to find the best hyperparameters.

In [31]:
# First install scikit-optimize
import sys
!{sys.executable} -m pip install scikit-optimize

You should consider upgrading via the '/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/bin/python -m pip install --upgrade pip' command.


In [32]:
# Import the necessary libraries
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args

In [33]:
# Redefine the search space from Random Search as dimensions
space = [
    Real(low=1e-1, high=2e3, prior='log-uniform', name='C'),
    Real(low=1e-4, high=1e-3, prior='log-uniform', name='gamma'),
    Categorical(categories=['rbf'], transform='identity', prior=None, name='kernel')
]

In [34]:
# Since Bayesian Optimization CV is currently broken, we will use the gp_minimize() function

@use_named_args(dimensions=space)
def func(**params):
    svc = SVC(**params)
    svc.fit(x_train, y_train)
    score = svc.score(x_test, y_test)
    # gp_minimize minimizes, so convert accuracy to inverse
    return 1 - score

In [35]:
# Start the search procress with the same number of runs
res = gp_minimize(func, dimensions=space, n_calls=num_runs, n_jobs=-1)

In [36]:
# Get the best parameters
res.x

[1830.2886345935435, 0.0003986729114551175, 'rbf']

In [37]:
# Evaluate model
bo_model = SVC(**dict(zip(['C', 'gamma', 'kernel'], res.x)))
bo_model.fit(x_train, y_train)
bo_model.score(x_test, y_test)

0.65

### **Bonus:**
Perform the same optimization on the dataset, but this time use the MLPClassifier.

In [38]:
from sklearn.neural_network import MLPClassifier

- What are problems you run into?
- What can you do to solve them?
- What is the best accuracy you can achieve with a MLP model?

In [39]:
# Define the search space
space_mlp = [
    Real(low=1e-4, high=1e0, prior='log-uniform', name='learning_rate_init'),
    Categorical(categories=['constant', 'invscaling', 'adaptive'], transform='identity', prior=None, name='learning_rate'),
    Real(low=1e-4, high=1e-2, prior='log-uniform', name='alpha'),
    Categorical(categories=['identity', 'logistic', 'tanh', 'relu'], transform='identity', prior=None, name='activation'),
    Categorical(categories=['lbfgs', 'sgd', 'adam'], transform='identity', prior=None, name='solver'),
    Integer(low=1, high=4, name='num_layers'),
    Integer(low=24, high=128, name='n1'),
    Integer(low=24, high=128, name='n2'),
    Integer(low=24, high=128, name='n3'),
    Integer(low=24, high=128, name='n4'),
]
# Define the target function
@use_named_args(dimensions=space_mlp)
def func_mlp(**params):
    num_layers = params['num_layers']
    neurons_in_layer = [params['n1'], params['n2'], params['n3'], params['n4']]
    params['hidden_layer_sizes'] = tuple([i for n, i in zip(range(num_layers), neurons_in_layer)])

    del params['num_layers']
    del params['n1']
    del params['n2']
    del params['n3']
    del params['n4']

    mlp = MLPClassifier(**params)
    mlp.fit(x_train, y_train)
    score = mlp.score(x_test, y_test)
    # gp_minimize minimizes, so convert accuracy to inverse
    return 1 - score
# Perform the minimization
res_mlp = gp_minimize(func_mlp, dimensions=space_mlp, n_calls=40, n_jobs=-1)

/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/nuls/.local/share/virtualenvs/bi_deep_learning-M-dGzlNK/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning:

In [40]:
1 - res_mlp.fun

0.675

In [41]:
res_mlp.x

[0.12621151482546025,
 'constant',
 0.0032137365160487008,
 'relu',
 'lbfgs',
 4,
 128,
 40,
 49,
 115]

In [51]:
params_dict = dict(zip(['learning_rate_init', 'learning_rate', 'alpha', 'activation', 'solver', 'num_layers', 'n1', 'n2', 'n3', 'n4'], res_mlp.x))
params_dict

{'learning_rate_init': 0.12621151482546025,
 'learning_rate': 'constant',
 'alpha': 0.0032137365160487008,
 'activation': 'relu',
 'solver': 'lbfgs',
 'num_layers': 4,
 'n1': 128,
 'n2': 40,
 'n3': 49,
 'n4': 115}

In [53]:
# Perform adaptation from above
num_layers = params_dict['num_layers']
neurons_in_layer = [params_dict['n1'], params_dict['n2'], params_dict['n3'], params_dict['n4']]
params_dict['hidden_layer_sizes'] = tuple([i for n, i in zip(range(num_layers), neurons_in_layer)])

del params_dict['num_layers']
del params_dict['n1']
del params_dict['n2']
del params_dict['n3']
del params_dict['n4']

In [54]:
mlp_model = MLPClassifier(**params_dict)
mlp_model.fit(x_train, y_train)
mlp_model.score(x_test, y_test)

0.55

In [ ]:
# We overfit the training set with our HPO